## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 2
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_2


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Team  games_played  season  offense_points_per_game  \
0  Las Vegas Raiders             2    2021                     29.5   
1  Arizona Cardinals             2    2021                     36.0   

   offense_passing_AVG  offense_passing_YDS/G  offense_passing_RTG  \
0                  8.8                  391.0                104.1   
1                 10.1                  325.5                121.5   

   offense_receiving_AVG  offense_rushing_YDS/G  defense_points_per_game  ...  \
0                   13.2                   67.0                     22.0  ...   
1                   13.8                  119.5                     23.0  ...   

   defense_receiving_AVG  defense_rushing_YDS/G  defense_passing_SYL_per_game  \
0                   11.5                  114.0                          10.5   
1                   10.5                  131.5                          29.0   

   offense_downs_Third Downs_ATT_per_game  \
0                                    14.0   
1                                    11.0   

   defense_downs_Third Downs_ATT_per_game  \
0                                    12.0   
1                                    12.0   

   defense_downs_Fourth Downs_ATT_per_game  defense_pass_to_rush_ratio  \
0                                      1.5                    1.458333   
1                                      1.5                    1.387755   

   defense_downs_First Downs_penalty_ratio  offense_passing_TD_per_game  \
0                                 0.054054                          2.0   
1                                 0.025641                          3.5   

   offense_pass_TD_to_INT  
0                4.000000  
1                2.333333  

[2 rows x 21 columns]

In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

Team  season week_marker   pred_rf  pred_gbm  pred_lgb  \
15      Los Angeles Rams    2021      week_2  0.354841  0.405177  0.532263   
12  Tampa Bay Buccaneers    2021      week_2  0.272897  0.233139  0.541960   
1      Arizona Cardinals    2021      week_2  0.307535  0.281552  0.332726   

    pred_catboost  pred_mean  
15       0.446773   0.434764  
12       0.359745   0.351935  
1        0.356962   0.319694

In [7]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [8]:
predictions

Team  season week_marker   pred_rf  pred_gbm  pred_lgb  \
15      Los Angeles Rams    2021      week_2  0.354841  0.405177  0.532263   
12  Tampa Bay Buccaneers    2021      week_2  0.272897  0.233139  0.541960   
1      Arizona Cardinals    2021      week_2  0.307535  0.281552  0.332726   
8     Kansas City Chiefs    2021      week_2  0.293002  0.245714  0.402016   
20         Buffalo Bills    2021      week_2  0.236373  0.229076  0.384719   
13     Carolina Panthers    2021      week_2  0.249194  0.244635  0.266986   
10      Seattle Seahawks    2021      week_2  0.292066  0.220844  0.234448   
14   Philadelphia Eagles    2021      week_2  0.205542  0.238732  0.223039   
6         Denver Broncos    2021      week_2  0.242620  0.240240  0.173042   
0      Las Vegas Raiders    2021      week_2  0.261861  0.182420  0.192600   
17   San Francisco 49ers    2021      week_2  0.234525  0.155483  0.145640   
5      Minnesota Vikings    2021      week_2  0.236990  0.145291  0.146187   
23    Cincinnati Bengals    2021      week_2  0.163339  0.122617  0.225822   
7       Cleveland Browns    2021      week_2  0.210522  0.128296  0.114293   
16        Houston Texans    2021      week_2  0.198569  0.117567  0.116523   
2       Baltimore Ravens    2021      week_2  0.177370  0.104925  0.133448   
31    New Orleans Saints    2021      week_2  0.109353  0.081802  0.065830   
22  New England Patriots    2021      week_2  0.128946  0.093533  0.050942   
4   Los Angeles Chargers    2021      week_2  0.101073  0.069788  0.070800   
3         Dallas Cowboys    2021      week_2  0.082990  0.043662  0.065581   
28     Green Bay Packers    2021      week_2  0.069128  0.047333  0.043894   
18       New York Giants    2021      week_2  0.059375  0.039227  0.058590   
27   Pittsburgh Steelers    2021      week_2  0.066229  0.048838  0.042154   
11         Detroit Lions    2021      week_2  0.034922  0.036058  0.074070   
30        Miami Dolphins    2021      week_2  0.067585  0.049644  0.054572   
21            Washington    2021      week_2  0.047265  0.047365  0.042634   
25         New York Jets    2021      week_2  0.045026  0.036391  0.052882   
26  Jacksonville Jaguars    2021      week_2  0.038206  0.045442  0.042054   
29         Chicago Bears    2021      week_2  0.038160  0.032182  0.040979   
24       Atlanta Falcons    2021      week_2  0.008066  0.027404  0.037912   
9       Tennessee Titans    2021      week_2  0.003532  0.030478  0.040793   
19    Indianapolis Colts    2021      week_2  0.003720  0.025697  0.038077   

    pred_catboost  pred_mean  
15       0.446773   0.434764  
12       0.359745   0.351935  
1        0.356962   0.319694  
8        0.297650   0.309595  
20       0.264623   0.278697  
13       0.282494   0.260827  
10       0.259583   0.251735  
14       0.305102   0.243104  
6        0.284703   0.235151  
0        0.217858   0.213685  
17       0.245980   0.195407  
5        0.226872   0.188835  
23       0.237180   0.187240  
7        0.202338   0.163862  
16       0.191392   0.156013  
2        0.187906   0.150912  
31       0.220835   0.119455  
22       0.184935   0.114589  
4        0.165488   0.101787  
3        0.163638   0.088968  
28       0.194656   0.088753  
18       0.192400   0.087398  
27       0.175220   0.083110  
11       0.174662   0.079928  
30       0.145409   0.079302  
21       0.163256   0.075130  
25       0.152167   0.071616  
26       0.149793   0.068874  
29       0.161469   0.068198  
24       0.152649   0.056508  
9        0.149162   0.055991  
19       0.146689   0.053546